In [1]:
from bs4 import BeautifulSoup
import requests

from IPython.core.display import display, HTML

import pandas as pd

import re

import numpy as np

import datetime

In [2]:
#Retrieve breakdown of games from october to end of march

urlList = [
    'https://www.basketball-reference.com/leagues/NBA_2019_games-october.html',
    'https://www.basketball-reference.com/leagues/NBA_2019_games-november.html',
    'https://www.basketball-reference.com/leagues/NBA_2019_games-december.html',
    'https://www.basketball-reference.com/leagues/NBA_2019_games-january.html',
    'https://www.basketball-reference.com/leagues/NBA_2019_games-february.html',
    'https://www.basketball-reference.com/leagues/NBA_2019_games-march.html'
]
soupList = []
for url in urlList:
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    soupList.append(soup)
[soup1,soup2,soup3,soup4,soup5,soup6] = soupList

In [3]:
headerData = soup1.find(class_ = 'overthrow table_container').find_all('tr')[0].find_all('th')

In [4]:
column_headers = []

#get headers
for header in headerData:
    column_headers.append(header['data-stat'])
column_headers

['date_game',
 'game_start_time',
 'visitor_team_name',
 'visitor_pts',
 'home_team_name',
 'home_pts',
 'box_score_text',
 'overtimes',
 'attendance',
 'game_remarks']

In [5]:
fullDataList = []

for soup in soupList:
    cellData = soup.find(class_ = 'overthrow table_container').find_all('tr')
    for row in range(1,len(cellData)):
        rowList = []
        for cell in cellData[row].find_all(lambda tag: tag.name == 'td' or tag.name == 'th'):
            if cell.contents == []:
                value = ''
            elif cell.contents[0].name == 'a':
                if cell.contents[0].contents[0] == 'Box Score':
                    value = 'https://www.basketball-reference.com'+ cell.contents[0]['href']
                else:
                    value = cell.contents[0].contents[0]
            else:
                value = cell.contents[0]
            rowList.append(value)
        fullDataList.append(rowList)

cleanCellData = [x for x in fullDataList if x != []]

cleanCellData

[['Tue, Oct 16, 2018',
  '8:00p',
  'Philadelphia 76ers',
  '87',
  'Boston Celtics',
  '105',
  'https://www.basketball-reference.com/boxscores/201810160BOS.html',
  '',
  '18,624',
  ''],
 ['Tue, Oct 16, 2018',
  '10:30p',
  'Oklahoma City Thunder',
  '100',
  'Golden State Warriors',
  '108',
  'https://www.basketball-reference.com/boxscores/201810160GSW.html',
  '',
  '19,596',
  ''],
 ['Wed, Oct 17, 2018',
  '7:00p',
  'Milwaukee Bucks',
  '113',
  'Charlotte Hornets',
  '112',
  'https://www.basketball-reference.com/boxscores/201810170CHO.html',
  '',
  '17,889',
  ''],
 ['Wed, Oct 17, 2018',
  '7:00p',
  'Brooklyn Nets',
  '100',
  'Detroit Pistons',
  '103',
  'https://www.basketball-reference.com/boxscores/201810170DET.html',
  '',
  '20,332',
  ''],
 ['Wed, Oct 17, 2018',
  '7:00p',
  'Memphis Grizzlies',
  '83',
  'Indiana Pacers',
  '111',
  'https://www.basketball-reference.com/boxscores/201810170IND.html',
  '',
  '17,923',
  ''],
 ['Wed, Oct 17, 2018',
  '7:00p',
  'Miam

In [6]:
#Combine Header and Data to create intiial DF

df = pd.DataFrame(cleanCellData,columns = column_headers)

In [7]:
def date_change(row):
    t = datetime.datetime.strptime(row['date_game'].replace(',',''), "%a %b %d %Y")
    convert_date = t.strftime('%m/%d/%Y')
    return(convert_date)

df['Date'] = pd.to_datetime(df.apply(date_change,axis = 1))

In [8]:
nbaDict = {
'Atlanta Hawks': 'ATL',
'Brooklyn Nets': 'BRK',
'Boston Celtics': 'BOS',
'Charlotte Hornets': 'CHO',
'Chicago Bulls': 'CHI',
'Cleveland Cavaliers': 'CLE',
'Dallas Mavericks': 'DAL',
'Denver Nuggets': 'DEN',
'Detroit Pistons': 'DET',
'Golden State Warriors': 'GSW',
'Houston Rockets': 'HOU',
'Indiana Pacers': 'IND',
'Los Angeles Clippers': 'LAC',
'Los Angeles Lakers': 'LAL',
'Memphis Grizzlies': 'MEM',
'Miami Heat': 'MIA',
'Milwaukee Bucks': 'MIL',
'Minnesota Timberwolves': 'MIN',
'New Orleans Pelicans': 'NOP',
'New York Knicks': 'NYK',
'Oklahoma City Thunder': 'OKC',
'Orlando Magic': 'ORL',
'Philadelphia 76ers': 'PHI',
'Phoenix Suns': 'PHO',
'Portland Trail Blazers': 'POR',
'Sacramento Kings': 'SAC',
'San Antonio Spurs': 'SAS',
'Toronto Raptors': 'TOR',
'Utah Jazz': 'UTA',
'Washington Wizards': 'WAS'}

In [9]:
statColumns = ['vis FG','vis FGA','vis FG %','vis 3P','vis 3PA',
    'vis 3P%','vis FT','vis FTA','vis FT%','vis ORB',
    'vis DRB','vis TRB','vis AST','vis STL','vis BLK',
    'vis TOV','vis PF','home FG','home FGA','home FG %',
    'home 3P','home 3PA','home 3P%','home FT','home FTA',
    'home FT%','home ORB','home DRB','home TRB','home AST',
    'home STL','home BLK','home TOV','home PF']

In [10]:
def statRecorder(row):
    url1 = row['box_score_text']
    response1 = requests.get(url1)
    page1 = response1.text
    soup1 = BeautifulSoup(page1, "lxml")
    team_list = [row['visitor_team_name'],row['home_team_name']]
    statsList = []
    for team in team_list:
        nbaTeam = nbaDict[team]
        boxScoreLink = "box-{}-game-basic".format(nbaTeam)
        totalStats = soup1.find('table', id =boxScoreLink).find_all('tr')[-1].find_all('td')
        for cell in totalStats[1:-2]:
            statsList.append(cell.contents[0])
    return(statsList)

In [11]:
#Create new stats list incrementally and then concatenate after
stats_list = []

In [12]:
zeroBatch = df[0:1].apply(statRecorder,axis = 1)
for i in zeroBatch:
    stats_list.insert(0,i)

In [13]:
firstBatch = df[1:101].apply(statRecorder,axis = 1)
for i in firstBatch:
    stats_list.append(i)

In [14]:
secondBatch = df[101:201].apply(statRecorder,axis = 1)
for i in secondBatch:
    stats_list.append(i)

In [15]:
thirdBatch = df[201:301].apply(statRecorder,axis = 1)
for i in thirdBatch:
    stats_list.append(i)

In [16]:
fourthBatch = df[301:401].apply(statRecorder,axis = 1)
for i in fourthBatch:
    stats_list.append(i)

In [17]:
fifthBatch = df[401:501].apply(statRecorder,axis = 1)
for i in fifthBatch:
    stats_list.append(i)

In [18]:
sixthBatch = df[501:548].apply(statRecorder,axis = 1)
for i in sixthBatch:
    stats_list.append(i)

In [31]:
df[545:548]

,date_game,game_start_time,visitor_team_name,visitor_pts,home_team_name,home_pts,box_score_text,overtimes,attendance,game_remarks,Date
545,"Mon, Dec 31, 2018",8:00p,Minnesota Timberwolves,114,New Orleans Pelicans,123,https://www.basketball-reference.com/boxscores...,,"14,904",,2018-12-31
546,"Mon, Dec 31, 2018",8:00p,Dallas Mavericks,102,Oklahoma City Thunder,122,https://www.basketball-reference.com/boxscores...,,"18,203",,2018-12-31
547,"Mon, Dec 31, 2018",9:00p,Golden State Warriors,132,Phoenix Suns,109,https://www.basketball-reference.com/boxscores...,,"16,906",,2018-12-31


In [19]:
nbaDictLower = {
'Atlanta Hawks': 'atl',
'Brooklyn Nets': 'brk',
'Boston Celtics': 'bos',
'Charlotte Hornets': 'cho',
'Chicago Bulls': 'chi',
'Cleveland Cavaliers': 'cle',
'Dallas Mavericks': 'dal',
'Denver Nuggets': 'den',
'Detroit Pistons': 'det',
'Golden State Warriors': 'gsw',
'Houston Rockets': 'hou',
'Indiana Pacers': 'ind',
'Los Angeles Clippers': 'lac',
'Los Angeles Lakers': 'lal',
'Memphis Grizzlies': 'mem',
'Miami Heat': 'mia',
'Milwaukee Bucks': 'mil',
'Minnesota Timberwolves': 'min',
'New Orleans Pelicans': 'nop',
'New York Knicks': 'nyk',
'Oklahoma City Thunder': 'okc',
'Orlando Magic': 'orl',
'Philadelphia 76ers': 'phi',
'Phoenix Suns': 'pho',
'Portland Trail Blazers': 'por',
'Sacramento Kings': 'sac',
'San Antonio Spurs': 'sas',
'Toronto Raptors': 'tor',
'Utah Jazz': 'uta',
'Washington Wizards': 'was'}

In [20]:
def statRecorder2(row):
    url1 = row['box_score_text']
    response1 = requests.get(url1)
    page1 = response1.text
    soup1 = BeautifulSoup(page1, "lxml")
    team_list = [row['visitor_team_name'],row['home_team_name']]
    statsList = []
    for team in team_list:
        nbaTeam = nbaDictLower[team]
        boxScoreLink = "box_{}_basic".format(nbaTeam)
        totalStats = soup1.find('table', id =boxScoreLink).find_all('tr')[-1].find_all('td')
        for cell in totalStats[1:-2]:
            statsList.append(cell.contents[0])
    return(statsList)

In [32]:
seventhBatch = df[548:701].apply(statRecorder,axis = 1)
for i in seventhBatch:
    stats_list.append(i)

In [33]:
eighthBatch = df[701:801].apply(statRecorder,axis = 1)
for i in eighthBatch:
    stats_list.append(i)

In [34]:
ninthBatch = df[801:901].apply(statRecorder,axis = 1)
for i in ninthBatch:
    stats_list.append(i)

In [35]:
tenthBatch = df[901:1001].apply(statRecorder,axis = 1)
for i in tenthBatch:
    stats_list.append(i)

In [36]:
eleventhBatch = df[1001:1152].apply(statRecorder,axis = 1)
for i in eleventhBatch:
    stats_list.append(i)

In [37]:
#Create new pd with stats list and stats columns
statsdf = pd.DataFrame(stats_list,columns = statColumns)
statsdf.head()

,vis FG,vis FGA,vis FG %,vis 3P,vis 3PA,vis 3P%,vis FT,vis FTA,vis FT%,vis ORB,...,home FTA,home FT%,home ORB,home DRB,home TRB,home AST,home STL,home BLK,home TOV,home PF
0,34,87,.391,5,26,.192,14,23,.609,6,...,14,.714,12,43,55,21,7,5,14,20
1,33,91,.363,10,37,.270,24,37,.649,16,...,18,.944,17,41,58,28,7,7,21,29
2,42,85,.494,14,34,.412,15,20,.750,11,...,22,.636,9,32,41,21,8,9,11,19
3,40,82,.488,5,27,.185,15,22,.682,5,...,22,.864,14,32,46,21,5,5,14,20
4,25,84,.298,10,29,.345,23,28,.821,7,...,13,.538,13,44,57,29,2,7,20,24


In [38]:
#concatenate original df with stats df
resultdf = pd.concat([df, statsdf], axis=1)
resultdf.head()

,date_game,game_start_time,visitor_team_name,visitor_pts,home_team_name,home_pts,box_score_text,overtimes,attendance,game_remarks,...,home FTA,home FT%,home ORB,home DRB,home TRB,home AST,home STL,home BLK,home TOV,home PF
0,"Tue, Oct 16, 2018",8:00p,Philadelphia 76ers,87,Boston Celtics,105,https://www.basketball-reference.com/boxscores...,,"18,624",,...,14,.714,12,43,55,21,7,5,14,20
1,"Tue, Oct 16, 2018",10:30p,Oklahoma City Thunder,100,Golden State Warriors,108,https://www.basketball-reference.com/boxscores...,,"19,596",,...,18,.944,17,41,58,28,7,7,21,29
2,"Wed, Oct 17, 2018",7:00p,Milwaukee Bucks,113,Charlotte Hornets,112,https://www.basketball-reference.com/boxscores...,,"17,889",,...,22,.636,9,32,41,21,8,9,11,19
3,"Wed, Oct 17, 2018",7:00p,Brooklyn Nets,100,Detroit Pistons,103,https://www.basketball-reference.com/boxscores...,,"20,332",,...,22,.864,14,32,46,21,5,5,14,20
4,"Wed, Oct 17, 2018",7:00p,Memphis Grizzlies,83,Indiana Pacers,111,https://www.basketball-reference.com/boxscores...,,"17,923",,...,13,.538,13,44,57,29,2,7,20,24


In [39]:
#Output to csv for saving purposes
export_csv = resultdf.to_csv (r'C:\Users\jeromerufin\Desktop\Metis\stats_data.csv') #Don't forget to add '.csv' at the end of the path

In [40]:
resultdf.head()

,date_game,game_start_time,visitor_team_name,visitor_pts,home_team_name,home_pts,box_score_text,overtimes,attendance,game_remarks,...,home FTA,home FT%,home ORB,home DRB,home TRB,home AST,home STL,home BLK,home TOV,home PF
0,"Tue, Oct 16, 2018",8:00p,Philadelphia 76ers,87,Boston Celtics,105,https://www.basketball-reference.com/boxscores...,,"18,624",,...,14,.714,12,43,55,21,7,5,14,20
1,"Tue, Oct 16, 2018",10:30p,Oklahoma City Thunder,100,Golden State Warriors,108,https://www.basketball-reference.com/boxscores...,,"19,596",,...,18,.944,17,41,58,28,7,7,21,29
2,"Wed, Oct 17, 2018",7:00p,Milwaukee Bucks,113,Charlotte Hornets,112,https://www.basketball-reference.com/boxscores...,,"17,889",,...,22,.636,9,32,41,21,8,9,11,19
3,"Wed, Oct 17, 2018",7:00p,Brooklyn Nets,100,Detroit Pistons,103,https://www.basketball-reference.com/boxscores...,,"20,332",,...,22,.864,14,32,46,21,5,5,14,20
4,"Wed, Oct 17, 2018",7:00p,Memphis Grizzlies,83,Indiana Pacers,111,https://www.basketball-reference.com/boxscores...,,"17,923",,...,13,.538,13,44,57,29,2,7,20,24


In [48]:
def win_percent(row):
    testurl = row['box_score_text']
    testresponse = requests.get(testurl)
    testpage = testresponse.text
    testsoup = BeautifulSoup(testpage,"lxml")
    wins_team = int(testsoup.findAll('div', text = re.compile("-"))[0].contents[0][0:].split('-')[0])
    losses_team = int(testsoup.findAll('div', text = re.compile("-"))[0].contents[0][0:].split('-')[1])
    win_pct_team = wins_team/(wins_team+losses_team)
    wins_opp = int(testsoup.findAll('div', text = re.compile("-"))[1].contents[0][0:].split('-')[0])
    losses_opp = int(testsoup.findAll('div', text = re.compile("-"))[1].contents[0][0:].split('-')[1])
    win_pct_opp = wins_opp/(wins_opp+losses_opp)
    win_list = [win_pct_team, win_pct_opp]
    return(win_list)

# win_first_batch = resultdf[0:101]
# win_second_batch = resultdf[101:201]
# win_third_batch = resultdf[201:301]
# win_fourth_batch = resultdf[301:401]
# win_fifth_batch = resultdf[401:501]
# win_sixth_batch = resultdf[501:601]
# win_seventh_batch = resultdf[601:701]
# win_eighth_batch = resultdf[701:801]
# win_ninth_batch = resultdf[801:901]
# win_tenth_batch = resultdf[901:1001]
# win_eleventh_batch = resultdf[1001:1152]

In [ ]:
win_final_list = []
for i in resultdf.apply(win_percent,axis=1):
    win_final_list.append(i)

In [ ]:
win_columns = ['Team_Win_Pct','Opp_Win_Pct']

win_pct_df = pd.DataFrame(win_final_list,columns = win_columns)

In [ ]:
resultdf = pd.concat([resultdf, win_pct_df], axis=1)

In [ ]:
#visitor stats
visitor_df = resultdf.iloc[:,np.r_[0:28,45]]

#home stats
home_df = resultdf.iloc[:,np.r_[0:2,4:6,2:4,6:11,28:45,47]]


In [ ]:
#Create bool indicator if the team is home or away
list_of_zeros = [0]*visitor_df.shape[0]
list_of_ones = [1]*home_df.shape[0]

away_bool_df = pd.DataFrame(list_of_zeros,columns = ['Home Team'])
home_bool_df = pd.DataFrame(list_of_ones,columns = ['Home Team'])

In [ ]:
#Add new column for binary home or away
visitor_df = pd.concat([visitor_df, away_bool_df], axis=1)
home_df = pd.concat([home_df, home_bool_df], axis=1)

In [ ]:
visitor_df.columns = ['date_game', 'game_start_time', 'team_name', 'team_pts',
       'opposing_team', 'opposing_team_pts', 'box_score_text', 'overtimes',
       'attendance', 'game_remarks', 'Date', 'FG', 'FGA', 'FG %',
       '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF','Home Team','Team_win_pct']


home_df.columns = ['date_game', 'game_start_time', 'team_name', 'team_pts',
       'opposing_team', 'opposing_team_pts', 'box_score_text', 'overtimes',
       'attendance', 'game_remarks', 'Date', 'FG', 'FGA', 'FG %',
       '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF','Home Team','Team_win_pct']

In [ ]:
#Combine dfs and clean-up unnecessary columns
frames = [visitor_df, home_df]
agg_data = pd.concat(frames,ignore_index=True)
agg_data['date_game'] = agg_data['Date']
del agg_data['Date']

In [ ]:
ordered_agg_data = agg_data.sort_values(by = ['team_name','date_game']).reset_index()

In [ ]:
ordered_agg_data.head()

In [ ]:
rolling_stat_data = agg_data.sort_values(by = ['team_name','date_game'])

testdata_grouped_rolling = rolling_stat_data.groupby('team_name')[['team_pts','opposing_team_pts','FG', 'FGA', 'FG %',
       '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%',
       'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK',
       'TOV', 'PF']].rolling(window=30, min_periods=1).mean().reset_index()

In [ ]:
del testdata_grouped_rolling['level_1']
del testdata_grouped_rolling['team_name']

In [ ]:
testdata_grouped_rolling.columns = ['Avg1 Pts','Avg1 Opp Pts','Avg1 FG','Avg1 FGA','Avg1 FG %',
       'Avg1 3P','Avg1 3PA','Avg1 3P%','Avg1 FT','Avg1 FTA','Avg1 FT%',
       'Avg1 ORB','Avg1 DRB','Avg1 TRB','Avg1 AST','Avg1 STL','Avg1 BLK',
       'Avg1 TOV','Avg1 PF']

In [ ]:
full_stats_data = pd.concat([ordered_agg_data ,testdata_grouped_rolling], axis=1)
full_stats_data.head(5)

In [ ]:
#Shift average stats so each row has the average of the games previously
full_stats_data[['Avg Pts','Avg Opp Pts','Avg FG','Avg FGA','Avg FG %',
       'Avg 3P','Avg 3PA','Avg 3P%','Avg FT','Avg FTA','Avg FT%',
       'Avg ORB','Avg DRB','Avg TRB','Avg AST','Avg STL','Avg BLK',
       'Avg TOV','Avg PF']] = full_stats_data.groupby('team_name')['Avg1 Pts','Avg1 Opp Pts','Avg1 FG','Avg1 FGA','Avg1 FG %',
       'Avg1 3P','Avg1 3PA','Avg1 3P%','Avg1 FT','Avg1 FTA','Avg1 FT%',
       'Avg1 ORB','Avg1 DRB','Avg1 TRB','Avg1 AST','Avg1 STL','Avg1 BLK',
       'Avg1 TOV','Avg1 PF'].apply(lambda grp: grp.shift(1))

In [ ]:
#Deleted non-shifted columns

full_stats_data = full_stats_data.drop(['Avg1 Pts','Avg1 Opp Pts','Avg1 FG','Avg1 FGA','Avg1 FG %',
       'Avg1 3P','Avg1 3PA','Avg1 3P%','Avg1 FT','Avg1 FTA','Avg1 FT%',
       'Avg1 ORB','Avg1 DRB','Avg1 TRB','Avg1 AST','Avg1 STL','Avg1 BLK',
       'Avg1 TOV','Avg1 PF'],axis =1)

In [ ]:
full_stats_data['Previous Game Data'] = (full_stats_data.groupby('team_name')['date_game']
                                            .apply(lambda grp: grp.shift(1)))

In [ ]:
#Do calculation for back to back games
full_stats_data['Time between games'] = full_stats_data['date_game'] - full_stats_data['Previous Game Data']

In [ ]:
full_stats_data['Win_pct'] = (full_stats_data.groupby('team_name')['Team_win_pct']
                                            .apply(lambda grp: grp.shift(1)))

In [ ]:
#Replicate df to join, so each line has team average and opposing team average

rep_full_stats_data = full_stats_data

rep_full_stats_data.head()

In [ ]:
pd.set_option('display.max_columns', 999)

In [ ]:
newDf = pd.merge(full_stats_data, rep_full_stats_data, left_on = ['date_game','opposing_team'],right_on = ['date_game','team_name'])

newDf.head()


In [ ]:
cleanNewDf1 = newDf.drop(newDf.loc[:,'Avg1 Pts_x':'Avg1 PF_x'],axis = 1)

In [ ]:
cleanNewDf1.head()

In [ ]:
cleandNewDf2 = cleanNewDf1.drop(cleanNewDf1.loc[:,'index_y':'game_remarks_y'],axis = 1)

In [ ]:
cleandNewDf3 = cleandNewDf2.drop(cleandNewDf2.loc[:,'Avg1 Pts_y':'Avg1 PF_y'],axis = 1)

In [ ]:
cleandNewDf3.head()

In [ ]:
cleandNewDf4 = cleandNewDf3.drop(cleandNewDf3.loc[:,'FG_x':'PF_x'],axis = 1)

In [ ]:
cleandNewDf4.head()

In [ ]:
cleandNewDf5 = cleandNewDf4.drop(cleandNewDf4.loc[:,'FG_y':'PF_y'],axis = 1)

In [ ]:
cleandNewDf = cleandNewDf5.drop(['index_x','overtimes_x','attendance_x'],axis =1)

In [ ]:
cleandNewDf.head()

In [ ]:
#Delete duplicates

dedupeData = cleandNewDf.drop_duplicates(subset ='box_score_text_x',keep = 'first')

dedupeData.head()

In [ ]:
dedupeData.columns

In [ ]:
dedupeData.columns = ['date_game', 'game_start_time', 'team_name', 'team_pts',
       'opposing_team', 'opposing_team_pts', 'box_score_text',
       'game_remarks', 'Team Home?', 'Avg Team Pts', 'Avg Opp Pts_x',
       'Avg Team FG', 'Avg Team FGA', 'Avg Team FG %', 'Avg Team 3P', 'Avg Team 3PA',
       'Avg Team 3P%', 'Avg Team FT', 'Avg Team FTA', 'Avg Team FT%', 'Avg Team ORB',
       'Avg Team DRB', 'Avg Team TRB', 'Avg Team AST', 'Avg Team STL', 'Avg Team BLK',
       'Avg Team TOV', 'Avg Team PF', 'Previous Game Data_x', 'Team Time between previous game',
       'Home Team_y', 'Avg Opp Pts', 'Avg Opp Pts_y',
       'Avg Opp FG', 'Avg Opp FGA', 'Avg Opp FG %', 'Avg Opp 3P', 'Avg Opp 3PA',
       'Avg Opp 3P%', 'Avg Opp FT', 'Avg Opp FTA', 'Avg Opp FT%', 'Avg Opp ORB',
       'Avg Opp DRB', 'Avg Opp TRB', 'Avg Opp AST', 'Avg Opp STL', 'Avg Opp BLK',
       'Avg Opp TOV', 'Avg Opp PF','Previous Game Data_y', 'Opp Time between previous game']

In [ ]:
Avg Opp Pts_x, 'Previous Game Data_x''Home Team_y', 'Avg Opp Pts_y', 'Previous Game Data_y',

In [ ]:
dedupeCleanDf = dedupeData.drop(['Avg Opp Pts_x','Previous Game Data_x','Home Team_y','Avg Opp Pts_y','Previous Game Data_y'],axis = 1)

In [ ]:
dedupeCleanDf.head()

In [ ]:
#Shift date to check if back-to-back game 

def tb2b(row):
    if pd.isnull(row['Team Time between previous game']):
        return(0)
    elif row['Team Time between previous game'].days == 1:
        return(1)
    return(0)

def ob2b(row):
    if pd.isnull(row['Opp Time between previous game']):
        return(0)
    elif row['Opp Time between previous game'].days == 1:
        return(1)
    return(0)
    
dedupeCleanDf['Team B2B?'] = dedupeCleanDf.apply(tb2b,axis = 1)
dedupeCleanDf['Opp B2B?'] = dedupeCleanDf.apply(ob2b,axis = 1)

In [ ]:
#East Team, West Team

nbaDictwest = {
'Atlanta Hawks': 0,
'Brooklyn Nets': 0,
'Boston Celtics': 0,
'Charlotte Hornets': 0,
'Chicago Bulls': 0,
'Cleveland Cavaliers': 0,
'Dallas Mavericks': 1,
'Denver Nuggets': 1,
'Detroit Pistons': 0,
'Golden State Warriors': 1,
'Houston Rockets': 1,
'Indiana Pacers': 0,
'Los Angeles Clippers': 1,
'Los Angeles Lakers': 1,
'Memphis Grizzlies': 1,
'Miami Heat': 0,
'Milwaukee Bucks': 0,
'Minnesota Timberwolves': 1,
'New Orleans Pelicans': 1,
'New York Knicks': 0,
'Oklahoma City Thunder': 1,
'Orlando Magic': 0,
'Philadelphia 76ers': 0,
'Phoenix Suns': 1,
'Portland Trail Blazers': 1,
'Sacramento Kings': 1,
'San Antonio Spurs': 1,
'Toronto Raptors': 0,
'Utah Jazz': 1,
'Washington Wizards': 0}